#BREAK ALL the things!

Welcome to destroying your first application... Literally!

In the code below we:
- Create a piece of C code (fine!) 
- We will compile it (fine!) 
- Try and run it (fine!)
- Try and run it when overflowing the buffer to break it... (BORK!)

It would seem Google will not let us continue with the exercise here...

In [0]:
# Creating the Program
!echo '#include <stdio.h>' > proggy.c
!echo 'int main () {' >> proggy.c
!echo '   char username[20];' >> proggy.c
!echo '   printf("Enter your name: ");' >> proggy.c
!echo '   scanf("%s", username);' >> proggy.c
!echo '   printf("Hello %s\n", username);' >> proggy.c
!echo '   printf("Program exited normally");' >> proggy.c
!echo '   return(0);' >> proggy.c
!echo '}' >> proggy.c
# Echo it all out:
!echo 'PROGRAM:'
!cat proggy.c
!echo ''
!echo 'COMPILE:'
!gcc proggy.c -o proggy
!echo ''
!echo 'PLAY:'
!echo 'tester' | ./proggy
!echo ''
!echo 'OVERFLOW:'
!python -c 'print("A"*35)' | ./proggy

### *** stack smashing detected ***
Yup! That's what we are trying to do with the help of our friend Python (and a little bit of C to prove there is no funny business going on!).

Sadly, we will need to continue this type of research from our own computers. To complete these exercises we will need:

- A Python run time environment - To run python
- an IDE that you are comfortable with - to write the code (no M$ Word docs please)
- gcc compiler - to compile our programs
- gdb to debug and see what is happening - To see what is happening

I have found a quick setup is to install docker and use:

`docker run  -ti ubuntu /bin/sh`

then run:

`apt-get update && apt-get -y python3 gdb nano vim emacs`

It does not give us the most friendly environment to work in but it has been reliable for demonstrating the concept.

The code in this workbook, from here on in is designed to be run locally or copied to your workstation and compiled.

If you get the Jupyter notebook running locally and connected to this console, it may work (untested).

## Basic Introduction to Computing

To understand how this attack vector is going to work for us we need to learn a little bit about how Computer memory and program execution works given the following code:

In [0]:
int main () {
   char username[20];

   printf("Enter your name: ");
   scanf("%s", username);

   printf("Hello %s\n", username);
   printf("Program exited normally!");
   return(0);
}

You may already see some of the bugs within the code that are ready to exploit.
The code takes a variable (about 20 characters long) and asks you to populate it for a friendly message before exiting.
We can see this ourselves if we copy and compile it. Save the code as `example1.c`
```
gcc example1.c example1 -fno-stack-protector
```
Will quickly compile the code and allow us to run it.
```
./example1
Enter your name: 
Hello
Program exited normally!
```

Now what is happening here - When we execute the program we are creating a "Stack" (The thingy that Google saw we were about to Smash!). A Stack is a part of Memory of the Computer allocated to run our application in. It has four layers and runs similar to:
- HEAP - The Dynamic Memory
- Stack - For the Function Calls and Local Variables
- Static/Globals Functions - can be used everywhere
- Code - The base instructions

Objects from each category above are stored together at memory addresses (of a set size) which can be seen if we decompile the binary we created for the exercise. 

With a decompiler we can see each instruction allocated an address:
`objdump -d ./example1`

On the left we can see the various memory locations. We won't go too much into the output here as it will differ depending on your setup.

We don't need to know too much more (at this stage - However, it is always good to be curious).

## Actually Smashing the Stack

If we are this far then you should have all the needed tools to delve deeper. Let's have a look at what happens when we attack our Binary. let's load up out Binary in a debugger:
`gdb ./example1`
This loads us into the debugger if we issues:
`(gdb) run`
We start the Program and it executes as normal. However, in the code we copied above we only allocated `[20]` to the buffer to hold our Name...

Hmm... I wonder what would happen if we had a REALLY long name? Don't you? 😹

```
Enter your name: ddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddd
Hello ddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddddd

Thread 3 received signal SIGABRT, Aborted.
0x00007fff6b9122c6 in ?? () from /usr/lib/system/libsystem_kernel.dylib
```
Whoops! We did it again! I got SIGABRT because the computer running the above had some stack protection and prevented the attack happening but you may get SIGSEGV which shows that we have managed to mess with the guts of the program during execution and it is unable to continue running. 

With a bit of knowledge about how the application writes itself and it's internals to memory we could have a little more fun with this little proggy...

To make life easier on ourselves let's quickly rewrite the program to work with standard input so we can pipe variables to it. 

This will make research a lot easier for us - call this `example2.c`:

In [0]:
#include <stdio.h>
#include <string.h>

int main(int argc, char** argv)
{
	char whatever[20];
	strcpy(whatever, argv[1]);

	return 0;
}

The code now takes an argument and tries to dump it into a 20 character variable without any care for what it might be given.

Once again let's compile:

`gcc example2.c example2 -fno-stack-protector`

but now let's look at it in a debugged - introducing gdb:

```
gdb ./example2
(gdb) run $(python –c "print('\x12'*99)")
```
Luckily in our context we know how much we need to overflow the buffer with `[20]` but if we didn't we could use python to have worked that out for us with a simple `for` loop.

give it a try and see if you can probe for the different responses the binary gives us.

With our crashed program we can look at the application stack and we should see that we have managed to overwrite all memory locations with the same address.

use the `(gdb) info registers` command to inspect your registers and see the difference with different amounts of characters provided.

## Ok... So how is that harmful?
...Well in our little exercise, it is not! 

This was an exercise to get you comfortable and curious about the subject. Hopefully, encourage you to read a little more around the subject matter. However, in bigger programs, especially one's that "borrow" permissions of a more powerful users or accounts. This technique can be used to do all manner of interesting and unexpected things on a computer and this is why we need to ensure our software is patched and has been written securely to handle these conditions. 

You may have also noticed the insecure flag that we are using to compile our C code. This should never be done for real production code, this flag removes modern protection given to compiled binaries to better illustrate the concepts.

Some users of this guide may have discovered errors where the insecure code did not error how we needed it to get the full benefit of the exploit and that will happen more frequently as security holes get patched and these bugs get harder to make happen. 


Play with the binaries and tools that we have used
you can de-compile the binaries with `gbd` as well [here](https://darkdust.net/files/GDB%20Cheat%20Sheet.pdf) is a cheat sheet that shows off some of the functions. See if you can:
- show the registers
- list the functions of the program
- decompile the main function

